## LLM Fine-Tuning (polyglot-ko-1.3b + LoRA)

공개된 대규모 언어모델 **"polyglot-ko-1.3b"** 을 Foundation Model로 하여 PEFT LoRA 방법을 이용하여 파인튜닝하는 실습을 진행해 보겠습니다.  
- Task: Causal Language Model (Instruct Fine-Tuning)
- Foundation Model: “polyglot-ko-1.3b”
- Dataset: “KorAlpaca”
- Trainer: Huggingface PEFT/LoRA

### 0. Setup

In [ ]:
# MLP Suwon 설정 필요
import os

os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

proxies = {
'http': '75.17.107.42:8080',
'https': '75.17.107.42:8080'
}

In [ ]:
# MLP Suwon 설정 필요
import ssl

if hasattr(ssl, '_create_unverified_context'):
   ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# !pip install --user transformers, datasets, peft, accelerate

In [ ]:
!nvidia-smi

### 1. DataSet: KoAlpaca v1.1a

질문을 전달하면 답변하는 형태로 대규모 언어모델을 파인튜닝하기 위해, 학습 데이터셋을 준비하겠습니다.  
데이터셋은 **Instruction**(지시사항)과 **Output**(출력)의 쌍으로 구성되어 있습니다.  
KoAlpaca 데이터셋은 지식iN 기반의 질문-답변 데이터셋이며, https://huggingface.co/datasets/beomi/KoAlpaca-v1.1a 에 공개되어 있습니다.

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

data

파인튜닝을 위하여 "### 질문: ..... \n\n ### 답변: ..... <|endoftext|>" 형태로 변환합니다.

In [ ]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

In [ ]:
data['train'][0]['text']

### 2. Model Loading

Foundation LLM "EleutherAI/polyglot-ko-1.3b" 모델을 로딩합니다.  
- Large-scale Korean Autoregressive LM
- Trained on 863 GB (213 billion tokens)
- GPT-NeoX framework
- Evaluation on 5 downstream tasks

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "EleutherAI/polyglot-ko-1.3b"

# 다음 코드를 완성하세요!! (사전학습 모델에 사용된 Tokenizer 가져오기)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 다음 코드를 완성하세요!! (사전학습 모델을 기반으로 CausalLM 구성)
model = AutoModelForCausalLM.from_pretrained(model_id)

Polyglot Model 학습에 사용된 토크나이저를 이용하여 파인튜닝 데이터를 Preprocessing 합니다. 

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

### 3. PEFT: LoRA

LoraConfig 함수를 통해 주요 파인튜닝 파라미터를 설정하고, PEFT LoRA 파인튜닝이 가능한 형태로 모델을 구성합니다.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

LoRA 모델을 통해 학습 가능한 파라미터수(1,572,864)는 전체 모델 파라미터(1,333,383,168)의 0.1% 수준입니다.

In [ ]:
from peft import LoraConfig, get_peft_model

# 다음 코드를 완성하세요!! (LoraConfig의 r, lora_alpha 값 설정)
config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

### 4. Model Training

Huggingface의 Transformer Trainer 기반으로 PEFT LoRA 파인튜닝을 진행합니다.

In [ ]:
!nvidia-smi

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

# 다음 코드를 완성하세요!! (Trainer 설정: batch_size, max_steps)
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        max_steps=100, ## 초소량만 학습: 100 step에 약 5분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
    ),
    # DataCollatorForLanguageModeling은 마스크 언어 모델링(MLM)과 인과적 언어 모델링(CLM)을 모두 지원합니다. 
    # 기본적으로 MLM용 데이터를 제공하지만 mlm=False 인수를 설정하여 CLM으로 전환할 수 있습니다:
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

TextGenerationPipeline 함수를 이용하여 질문에 대한 답변을 생성해 보겠습니다.

In [ ]:
model.eval() # evaluation mode
model.config.use_cache = True

In [ ]:
from transformers import TextGenerationPipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

text_generation_pipeline = TextGenerationPipeline(
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device=device,
)

In [ ]:
# 다음 코드를 완성하세요!! (문장 생성에 필요한 파라미터 설정: temperature, top_k, do_sample, no_repeat_ngram_size)
def generate_text(prompt, max_length=256, temperature=0.5):
    prompt = f"### 질문: {prompt}\n\n### 답변:"
    generated_sequences = text_generation_pipeline(
        prompt,
        top_k=5, # The number of predictions to return
        num_return_sequences=1,
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        eos_token_id=2,
        pad_token_id=2, # tokenizer.eos_token_id
        max_length=max_length,
    )
 
    return generated_sequences[0]["generated_text"]

In [ ]:
generate_text('건강하게 살기 위한 세 가지 방법은?')